# #6. Recommender System

In [ ]:
import numpy as np
import pandas as pd
import datetime as datetime
# import dataload
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
tqdm.pandas()

# 0. Data Load

## 0-1. 구매이력 데이터

In [ ]:
buy_full = pd.read_csv('최종데이터/final_categorical_0622.csv', encoding='cp949')
buy_full['date'] = buy_full['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
buy_full['year'] = buy_full['date'].apply(lambda x : x.year)
buy_full['month'] = buy_full['date'].apply(lambda x : x.month)
buy_full['day'] = buy_full['date'].apply(lambda x : x.day)

## 0-2. 날씨와 상관관계 높은 상품 선택

In [ ]:
adf_corr = pd.read_csv('최종데이터/adf_corr_group_0617.csv', encoding='cp949')
buy_full = buy_full.merge(adf_corr[['sm_cat', 'group']], on='sm_cat')
data = buy_full[(buy_full.group=='정상_high')|(buy_full.group=='비정상_high')].reset_index(drop=True)

# 1. Recommender System  
* Item based Collaborative Filtering 기법 사용

## 1-1. 성/연령/조회월/타겟상품/날씨별 추천상품 도출

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

class cf_item:
    def __init__(self, data, target, age, sex, month):
        self.data = data
        self.target = target
        self.age = age
        self.sex = sex
        self.month = month

        # age
        if self.age == 20:
            final_data = self.data[self.data['age']==20]
        elif self.age == 30:
            final_data = self.data[self.data['age']==30]
        elif self.age == 40:
            final_data = self.data[self.data['age']==40]
        elif self.age == 50:
            final_data = self.data[self.data['age']==50]
        else:
            final_data = self.data[self.data['age']==50]        
        # sex
        if self.sex == 'F':
            final_data = final_data[final_data['sex']=='F']
        elif self.sex == 'M':
            final_data = final_data[final_data['sex']=='M']
        # month
        final_data = final_data[final_data['month']==self.month]

        # preprocessing
        final_data['date'] = final_data['date'].apply(lambda x : str(x)[:10])
        self.final_data = final_data

    def recommender(self, sample_data, number):
        # collaborative filtering
        item_user = pd.pivot_table(sample_data, values='qty', index='sm_cat', columns='date').fillna(0)
        cosine_similarity0 = cosine_similarity(item_user)
        item_collabor = pd.DataFrame(data=cosine_similarity0, index=item_user.index, columns=item_user.index)
        result = item_collabor[self.target].sort_values(ascending=False)[1:(number+1)]
        res = pd.DataFrame(result).reset_index(); res.columns = ['rec', 'sim']
        res['sm_cat'] = self.target; res['rank'] = np.argsort(-res['sim']) + 1
        return res

    def recommender_weather(self, number, rain, dust):
        global weather_data
        # 비안옴(0) & 미세먼지 좋음(0)
        if (rain == 0)&(dust == 0):
            weather_data = self.final_data.loc[(self.final_data['rn_day']==0)&(self.final_data['PM10']<81)]
        # 비안옴(0) & 미세먼지 나쁨(1)    
        elif (rain == 0)&(dust == 1):
            if self.final_data.loc[(self.final_data['rn_day']==0)&(self.final_data['PM10']>=81)].date.nunique() <= 3:
                weather_data = self.final_data.loc[(self.final_data['rn_day']==0)&(self.final_data['PM10']>=51)]
        # 비옴(1) & 미세먼지 좋음(0)    
        elif (rain == 1)&(dust == 0):
            weather_data = self.final_data.loc[(self.final_data['rn_day']>0)&(self.final_data['PM10']<81)]
        # 비옴(1) & 미세먼지 나쁨(1)    
        elif (rain == 1)&(dust == 1):
            if self.final_data.loc[(self.final_data['rn_day']>0)&(self.final_data['PM10']>=81)].date.nunique() <= 3:
                weather_data = self.final_data.loc[(self.final_data['rn_day']>0)&(self.final_data['PM10']>=51)]
        nrows = len(weather_data.date.unique())
        
        if nrows <= 3:
            # print('오늘 날씨에 해당되는 데이터가 부족하므로 날씨를 고려하지 않고 아이템을 추천해드리겠습니다 ㅠ_ㅠ')
            res1 = self.recommender(self.final_data, number)
            if rain==0:
                res1['rain'] = '맑은 날'
            elif rain==1:
                res1['rain'] = '비오는 날'
            if dust==0:
                res1['dust'] = '미세먼지 좋은 날'
            elif dust==1:
                res1['dust'] = '미세먼지 나쁜 날'
            res1['age'] = self.age; res1['sex'] = self.sex; res1['month'] = self.month
            res1['sm_cat'] = self.target
            res1['text'] = '해당 날씨의 조건에 부합하는 데이터가 부족하여, 성연령 정보만을 고려한 추천 결과입니다.'
            res1 = res1[['age', 'sex', 'month', 'sm_cat', 'rain', 'dust', 'rank', 'rec', 'sim', 'text']]
            return res1

        else:
            # collaborative filtering
            res1 = self.recommender(weather_data, number)
            if rain==0:
                res1['rain'] = '맑은 날'
            elif rain==1:
                res1['rain'] = '비오는 날'
            if dust==0:
                res1['dust'] = '미세먼지 좋은 날'
            elif dust==1:
                res1['dust'] = '미세먼지 나쁜 날'
            res1['age'] = self.age; res1['sex'] = self.sex; res1['month'] = self.month
            res1['text'] = ''
            res1 = res1[['age', 'sex', 'month', 'sm_cat', 'rain', 'dust', 'rank', 'rec', 'sim', 'text']]
            return res1

In [ ]:
NUMBER = 10
final_rec = pd.DataFrame()

for age in data.age.unique():
    for sex in data.sex.unique():
        print('========{}{}========'.format(age,sex))
        for sm_cat in data.sm_cat.unique():
            for month in range(1, 13):
                cf = cf_item(data, sm_cat, age, sex, month)
                # 연령 텍스트
                age_text = '{}대'.format(age)
                # 성별 텍스트
                if sex == 1:
                    sex_text = '여성'
                else:
                    sex_text = '남성'
                try:
                    final_rec = pd.concat([final_rec, cf.recommender_weather(NUMBER, 0, 0)], axis=0)
                except:
                    stop = pd.DataFrame({'age' : [age_text] * NUMBER, 'sex' : [sex_text] * NUMBER, 'month' : [month] * NUMBER,
                                         'sm_cat' : [sm_cat] * NUMBER, 'rain' : ['맑은 날'] * NUMBER,
                                         'dust' : ['미세먼지 좋은 날'] * NUMBER, 'rank' : range(1,(1+NUMBER)),
                                         'text' : '해당 날씨의 조건에 부합하는 데이터가 부족하여, 성연령 정보만을 고려한 추천 결과입니다.'
                                         })
                    final_rec = pd.concat([final_rec, stop], axis=0)

                try:
                    final_rec = pd.concat([final_rec, cf.recommender_weather(10, 0, 1)], axis=0)
                except:
                    stop = pd.DataFrame({'age' : [age_text] * NUMBER, 'sex' : [sex_text] * NUMBER, 'month' : [month] * NUMBER,
                                         'sm_cat' : [sm_cat] * NUMBER, 'rain' : ['맑은 날'] * NUMBER,
                                         'dust' : ['미세먼지 나쁜 날'] * NUMBER, 'rank' : range(1,(1+NUMBER)),
                                         'text' : '해당 날씨의 조건에 부합하는 데이터가 부족하여, 성연령 정보만을 고려한 추천 결과입니다.'
                                         })
                    final_rec = pd.concat([final_rec, stop], axis=0)

                try:
                    final_rec = pd.concat([final_rec, cf.recommender_weather(10, 1, 0)], axis=0)
                except:
                    stop = pd.DataFrame({'age' : [age_text] * NUMBER, 'sex' : [sex_text] * NUMBER, 'month' : [month] * NUMBER,
                                         'sm_cat' : [sm_cat] * NUMBER, 'rain' : ['비오는 날'] * NUMBER,
                                         'dust' : ['미세먼지 좋은 날'] * NUMBER, 'rank' : range(1,(1+NUMBER)),
                                         'text' : '해당 날씨의 조건에 부합하는 데이터가 부족하여, 성연령 정보만을 고려한 추천 결과입니다.'
                                         })
                    final_rec = pd.concat([final_rec, stop], axis=0)

                try:
                    final_rec = pd.concat([final_rec, cf.recommender_weather(10, 1, 1)], axis=0)
                except:
                    stop = pd.DataFrame({'age' : [age_text] * NUMBER, 'sex' : [sex_text] * NUMBER, 'month' : [month] * NUMBER,
                                         'sm_cat' : [sm_cat] * NUMBER, 'rain' : ['비오는 날'] * NUMBER,
                                         'dust' : ['미세먼지 나쁜 날'] * NUMBER, 'rank' : range(1,(1+NUMBER)),
                                         'text' : '해당 날씨의 조건에 부합하는 데이터가 부족하여, 성연령 정보만을 고려한 추천 결과입니다.'
                                         })
                    final_rec = pd.concat([final_rec, stop], axis=0)

In [ ]:
final_rec.to_csv('최종데이터/final_rec0625.csv', encoding='cp949')